In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'G'
df = pd.read_excel("Retail2.xlsx")

In [ ]:
y = df.set_index(['Date'])
y.head(5)

In [ ]:
y.plot(figsize=(19, 4))
plt.show()

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(y, model='additive')
fig = decomposition.plot()
plt.show()

Давайте использовать SARIMA. Обозначения моделей SARIMA(p, d, q).(P,D,Q)m. Эти три параметра учитывают сезонность, тенденцию и шум в данных

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
for param in pdq:
for param_seasonal in seasonal_pdq:
    try:
    mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
    results = mod.fit()
    print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
    except: 
continue

AIC (Информационный критерий Akaike) - это оценка относительного качества статистика модели для заданного набора данных. Дан набор моделей для данных, AIC оценивает качество каждой модели относительно каждой из других моделей. Низкий AIC цените лучшее. Наш вывод предполагает, что SARIMAX(0, 0, 1)x(1, 1, 1, 12)AIC значение 223,43 - лучшая комбинация, поэтому мы должны считать это оптимальным вариантом.

In [ ]:
mod = sm.tsa.statespace.SARIMAX(y,
 order=(0, 0, 1),
 seasonal_order=(1, 1, 1, 12),
 enforce_stationarity=False,
 enforce_invertibility=False)
results = mod.fit()
print(results.summary().tables[1])

In [ ]:
results.plot_diagnostics(figsize=(18, 8))
plt.show()

С помощью приведенной выше диагностики мы можем визуализировать важную информацию в виде распределения и функции автоматической корреляции ACF (коррелограммы). Значения вверх “0” имеют некоторую корреляцию с данными временных рядов. Значения, близкие к “1”, демонстрируют самую сильную корреляцию.

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2018-06-01'), dynamic=False)
pred_ci = pred.conf_int()
ax = y['2015':].plot(label='наблюдается')
pred.predicted_mean.plot(ax= ax, label= 'Прогноз на один шаг вперед', alpha = .7, figsize=(14, 4))
ax.fill_between(pred_ci.index,
 pred_ci.iloc[:, 0],
 pred_ci.iloc[:, 1], color= 'k', alpha =.2)
ax.set_xlabel('Дата')
ax.set_ylabel('Retail_sold')
plt.legend()
plt.show()

Этот шаг состоит в сравнении истинных значений с прогнозируемыми прогнозами. Наши прогнозы очень хорошо соответствуют истинным значениям. Команда “pred = results.get_prediction(start= pd.to_datetime(‘2018-06-01’)” определяет период, который вы бы прогнозировали при сравнении с истинными данными.

In [ ]:
y_forecasted = pred.predicted_mean
y_truth = y['2018-06-01':]
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error is {}'.format(round(mse, 2)))
print('The Root Mean Squared Error is {}'.format(round(np.sqrt(mse), 2)))

In [ ]:
pred_uc = results.get_forecast(steps=12)
pred_ci = pred_uc.conf_int()
ax = y.plot(label='observed', figsize=(14, 4))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
 pred_ci.iloc[:, 0],
 pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Sales')
plt.legend()
plt.show()

In [ ]:
pred_uc = results.get_forecast(steps=12)
pred_ci = pred_uc.conf_int()
ax = y.plot(label='observed', figsize=(14, 4))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
 pred_ci.iloc[:, 0],
 pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Sales')
plt.legend()
plt.show()

Здесь мы прогнозируем продажи на следующие 12 месяцев. Этот параметр можно изменить в строке “pred_uc = results.get_forecast(шаги = 12)” кода.

In [ ]:
y_forecasted = pred.predicted_mean
y_forecasted.head(12)

In [ ]:
y_truth.head(12)

In [ ]:
pred_ci.глава(24)

In [ ]:
forecast = pred_uc.predicted_mean
forecast.head(12)

Наконец, здесь у нас есть прогноз продаж на следующие 12 месяцев!